In [175]:
from datetime import datetime
from elasticsearch import Elasticsearch

In [176]:
es = Elasticsearch(['35.194.152.180:9200'], timeout=30)

In [171]:
help(es.bulk)

Help on method bulk in module elasticsearch.client:

bulk(body, index=None, doc_type=None, params=None) method of elasticsearch.client.Elasticsearch instance
    Perform many index/delete operations in a single API call.
    
    See the :func:`~elasticsearch.helpers.bulk` helper function for a more
    friendly API.
    `<http://www.elastic.co/guide/en/elasticsearch/reference/current/docs-bulk.html>`_
    
    :arg body: The operation definition and data (action-data pairs),
        separated by newlines
    :arg index: Default index for items which don't provide one
    :arg doc_type: Default document type for items which don't provide one
    :arg _source: True or false to return the _source field or not, or
        default list of fields to return, can be overridden on each sub-
        request
    :arg _source_exclude: Default list of fields to exclude from the
        returned _source field, can be overridden on each sub-request
    :arg _source_include: Default list of fields to

In [173]:
es.bulk["errors"]

TypeError: 'method' object is not subscriptable

In [99]:
doc = {
    'author': 'kimchy',
    'text': 'Elasticsearch: cool. bonsai cool.',
    'timestamp': datetime.now(),
}
res = es.index(index="t-index", doc_type='tweet', id=1, body=doc)
print(res['result'])

created


In [183]:
empty = {
}
res = es.index(index="t", doc_type='e', body=empty)
print(res['result'])

created


In [184]:
es.delete(index="t",doc_type="e",id=1)

{'_index': 't',
 '_type': 'e',
 '_id': '1',
 '_version': 2,
 'result': 'deleted',
 '_shards': {'total': 2, 'successful': 1, 'failed': 0},
 '_seq_no': 1,
 '_primary_term': 1}

In [100]:
df = pd.DataFrame(np.random.randint(0,100,size=(100, 4)), columns=list('ABCD'))

In [149]:
INDEX="dataframe"
TYPE= "record"

In [148]:
json.dumps(df.to_dict(orient="records")[0], default=int)

'{"A": 80, "B": 64, "C": 94, "D": 90}'

In [146]:
str(df.to_dict(orient="records")[0])

"{'A': 80, 'B': 64, 'C': 94, 'D': 90}"

In [127]:
import json
for record in df.to_dict(orient="records"):
    print('{ "index" : { "_index" : "%s", "_type" : "%s" }}'% (INDEX, TYPE))
    print(json.dumps(record, default=int))

{ "index" : { "_index" : "dataframe", "_type" : "record" }}
{"A": 80, "B": 64, "C": 94, "D": 90}
{ "index" : { "_index" : "dataframe", "_type" : "record" }}
{"A": 43, "B": 47, "C": 42, "D": 17}
{ "index" : { "_index" : "dataframe", "_type" : "record" }}
{"A": 45, "B": 9, "C": 62, "D": 69}
{ "index" : { "_index" : "dataframe", "_type" : "record" }}
{"A": 20, "B": 36, "C": 48, "D": 43}
{ "index" : { "_index" : "dataframe", "_type" : "record" }}
{"A": 45, "B": 59, "C": 24, "D": 65}
{ "index" : { "_index" : "dataframe", "_type" : "record" }}
{"A": 17, "B": 38, "C": 90, "D": 33}
{ "index" : { "_index" : "dataframe", "_type" : "record" }}
{"A": 44, "B": 39, "C": 65, "D": 20}
{ "index" : { "_index" : "dataframe", "_type" : "record" }}
{"A": 68, "B": 42, "C": 9, "D": 63}
{ "index" : { "_index" : "dataframe", "_type" : "record" }}
{"A": 74, "B": 55, "C": 68, "D": 79}
{ "index" : { "_index" : "dataframe", "_type" : "record" }}
{"A": 67, "B": 82, "C": 46, "D": 88}
{ "index" : { "_index" : "datafr

In [160]:
def rec_to_actions(df):
    import json
    for record in df.to_dict(orient="records"):
        yield ('{ "index" : { "_index" : "%s", "_type" : "%s" }}'% (INDEX, TYPE))
        yield (json.dumps(record, default=int))

In [161]:
rec_to_actions(df)

<generator object rec_to_actions at 0x119e1dc50>

In [162]:
es

<Elasticsearch([{'host': '35.194.152.180', 'port': 9200}])>

In [163]:
if not es.indices.exists(INDEX):
    raise RuntimeError('index does not exists, use `curl -X PUT "localhost:9200/%s"` and try again'%INDEX)

r = es.bulk(rec_to_actions(df)) # return a dict

print(not r["errors"])

True


In [155]:
type(df)

pandas.core.frame.DataFrame

In [102]:
import pandas as pd
import numpy as np
from espandas import Espandas

In [43]:
df = (100 * pd.DataFrame(np.round(np.random.rand(100, 5), 2))).astype(int)
df.columns = ['A', 'B', 'C', 'D', 'E']
df['indexId'] = (df.index + 100).astype(str)

In [44]:
INDEX = 'foo_index'
TYPE = 'bar_type'
esp = Espandas()

In [45]:
esp.es_write(df, INDEX, TYPE)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/espandas/espandas.py:85: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  df = df.reindex_axis(sorted(df.columns), axis=1)


In [90]:
def pd_elastic(df):
    from espandas import Espandas
    df['indexId'] = (df.index).astype(str)
    INDEX = input('index name: ')
    TYPE = input('type name: ')
    esp = Espandas()
    esp.es_write(df, INDEX, TYPE)

In [24]:
df = pd.read_csv('f100.csv')
f100_t = df.head(15)
f100_t['indexId'] = (f100_t.index).astype(str)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [31]:
help(pd.core.indexes.range.RangeIndex.astype)

Help on function astype in module pandas.core.indexes.base:

astype(self, dtype, copy=True)
    Create an Index with values cast to dtypes. The class of a new Index
    is determined by dtype. When conversion is impossible, a ValueError
    exception is raised.
    
    Parameters
    ----------
    dtype : numpy dtype or pandas type
    copy : bool, default True
        By default, astype always returns a newly allocated object.
        If copy is set to False and internal requirements on dtype are
        satisfied, the original data is used to create a new Index
        or the original Index is returned.
    
        .. versionadded:: 0.19.0



In [25]:
INDEX = 'f100_t_index'
TYPE = 'f100_t_type'
esp = Espandas()
esp.es_write(f100_t, INDEX, TYPE)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/espandas/espandas.py:85: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  df = df.reindex_axis(sorted(df.columns), axis=1)


In [23]:
help(esp.es_write)

Help on method es_write in module espandas.espandas:

es_write(df, index, doc_type, index_name='indexId') method of espandas.espandas.Espandas instance
    Insert a Pandas DataFrame into ElasticSearch
    :param df: the DataFrame, must contain the column 'indexId' for a unique identifier
    :param index: the ElasticSearch index
    :param doc_type: the ElasticSearch doc_type



In [29]:
type(f100_t.index)

pandas.core.indexes.range.RangeIndex

In [46]:
df = pd.read_csv('rsi_t.csv')

In [84]:
df.head()

,es_date,raw_date,btc_volume,btc_price,RSI_6,RSI_9,RSI_15,indexId
0,2010-07-24T00:00:00Z,20100724,10400.000000,0.053,NaN,NaN,NaN,0
1,2011-03-01T00:00:00Z,20110301,79956.000000,0.930,NaN,NaN,NaN,1
2,2011-06-04T00:00:00Z,20110604,0.001000,17.830,NaN,NaN,NaN,2
3,2011-08-08T00:00:00Z,20110808,10770.515208,8.890,NaN,NaN,NaN,3
4,2012-06-22T00:00:00Z,20120622,0.031337,6.490,NaN,NaN,NaN,4


In [81]:
df['indexId'] = (df.index).astype(str)
INDEX = 'rsi_index'
TYPE = 'rsi_type'
esp = Espandas()
esp.es_write(df, INDEX, TYPE)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/espandas/espandas.py:85: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  df = df.reindex_axis(sorted(df.columns), axis=1)


In [164]:
help(es.bulk)

Help on method bulk in module elasticsearch.client:

bulk(body, index=None, doc_type=None, params=None) method of elasticsearch.client.Elasticsearch instance
    Perform many index/delete operations in a single API call.
    
    See the :func:`~elasticsearch.helpers.bulk` helper function for a more
    friendly API.
    `<http://www.elastic.co/guide/en/elasticsearch/reference/current/docs-bulk.html>`_
    
    :arg body: The operation definition and data (action-data pairs),
        separated by newlines
    :arg index: Default index for items which don't provide one
    :arg doc_type: Default document type for items which don't provide one
    :arg _source: True or false to return the _source field or not, or
        default list of fields to return, can be overridden on each sub-
        request
    :arg _source_exclude: Default list of fields to exclude from the
        returned _source field, can be overridden on each sub-request
    :arg _source_include: Default list of fields to